# Table of Contents
 <p><div class="lev1"><a href="#Distance-Decay-Weighting-Options"><span class="toc-item-num">1&nbsp;&nbsp;</span>Distance Decay Weighting Options</a></div><div class="lev2"><a href="#Define-Utility-Functions"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Define Utility Functions</a></div><div class="lev2"><a href="#Trial-Analysis"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Trial Analysis</a></div><div class="lev3"><a href="#Linear-Distance-Decay"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Linear Distance Decay</a></div><div class="lev3"><a href="#Exponential-Decay-with-Scale-Factor"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Exponential Decay with Scale Factor</a></div>

In [19]:
import numpy as np
import networkx as nx
import itertools
import math
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Distance Decay Weighting Options #

The main idea behind a approximate-nearest-neighbor interaction network is that most of the inter-community links are fairly local, i.e., span short geographic distances.  This should create a regional interaction network where innovations do not spread instantly across the region but instead pass through intermediate communites as they diffuse across a region, the presence of occasional long-distance connection notwithstanding.

The question is what kind of distance decay kernel and weighting scheme leads to randomly constructed networks.  Linear decay (i.e., simply scaling the distances into probabilities) leads to min/max/mean distances that are smaller than potential distances (i.e., distances between vertices in the graph whether there are any edges linking them), but not much smaller.  This notebook is an exploration of distance kernels to determine how to scale the decay within an abstract RTN model such that we end up with average realized edge distances that are only a fraction of the maximum distances possible.  

## Define Utility Functions ##

In [6]:
def build_distance_map_to_self(g):
    """
    Constructs twolevel map of distances from each node in the current slice, to other nodes in the same slice

    :param g:
    :return:
    """
    dist_map = dict()
    for n,d in g.nodes_iter(data=True):
        n_label = g.node[n]['label']
        dist_map[n_label] = dict()
        n_x = int(g.node[n]['xcoord'])
        n_y = int(g.node[n]['ycoord'])

        for l,d in g.nodes_iter(data=True):
            # we don't need self distances
            if l == n:
                continue
            l_label = g.node[l]['label']
            l_x = float(g.node[l]['xcoord'])
            l_y = float(g.node[l]['ycoord'])
            dist = math.sqrt(pow(l_y - n_y, 2) + pow(l_x - n_x, 2))
            dist_map[n_label][l_label] = dist

    return dist_map

In [7]:
def calculate_community_distance_statistics(g, ignore_actual=True):
    """
    Calculates both the min/max/average edge distance, taking each edge into account only once,
    and the possible values for these given all pairs of vertices whether linked or not.  There
    are enough returned statistics that the function returns a dict with keys:  min_actual,
    mean_potential, etc.

    For actual values, pass "ignore_actual=False".  This allows the function to be used on
    sets of vertices with geographic coordinates but before we actually wire the edges.

    :param g:
    :param: ignore_actual - boolean to not calculate statistics for actual edges, since we might not have any yet.
    :return: dict
    """
    actual_distances = []
    potential_distances = []

    nodes = g.nodes()
    for i,j in itertools.product(nodes, nodes):
        if i == j:
            continue
        i_x = float(g.node[i]['xcoord'])
        i_y = float(g.node[i]['ycoord'])
        j_x = float(g.node[j]['xcoord'])
        j_y = float(g.node[j]['ycoord'])
        dist = math.sqrt(pow(i_y - j_y, 2) + pow(i_x - j_x, 2))
        if g.has_edge(i,j):
            actual_distances.append(dist)
        potential_distances.append(dist)

    # log.debug("actual dist: %s", actual_distances)
    # log.debug("potential dist: %s", potential_distances)

    res = dict()
    if ignore_actual == False:
        res['actual_min'] = np.amin(actual_distances)
        res['actual_max'] = np.amax(actual_distances)
        res['actual_mean'] = np.mean(actual_distances)
        res['edge_density'] = float(len(actual_distances)) / float(len(potential_distances))
    res['potential_min'] = np.amin(potential_distances)
    res['potential_max'] = np.amax(potential_distances)
    res['potential_mean'] = np.mean(potential_distances)

    # log.debug("res: %s", res)
    return res


In [10]:
def build_weighted_node_lists_exponential_decay(node, dist_map, average_potential_dist, alpha=1.0):
    """
    Build a list of neighbors for the focal node, and a list of probability weights where
    the probability is linear in the inverse of distance (smaller distances equal larger weights)

    :param node:
    :param dist_map:
    :param average_potential_dist:
    :param alpha: scaling factor, governs how steep the decay curve is.
    :return:
    """
    node_list = []
    weight_list = []

    distances = []

    for label,d in dist_map[node].items():
        distances.append(d)
        node_list.append(label)

    n_distances = np.asarray(distances)
    # log.debug("distances: %s", n_distances)

    # raw exponential weights are exp(-distance[i]/average_potential_dist)
    raw_weights = np.exp(alpha * (n_distances * -1)/average_potential_dist)
    #log.debug("raw exponential weights: %s", raw_weights)

    total_weights = np.sum(raw_weights)

    scaled_weights = raw_weights / total_weights
    #log.debug("scaled exponential weights: %s", scaled_weights)

    return (node_list, scaled_weights)


In [36]:


def build_weighted_node_lists_linear_decay(node, dist_map, max_x_coord, max_y_coord):
    """
    Build a list of neighbors for the focal node, and a list of probability weights where
    the probability is linear in the inverse of distance (smaller distances equal larger weights)

    :param node:
    :param dist_map:
    :return:
    """
    node_list = []
    weight_list = []

    distances = []

    for label,d in dist_map[node].items():
        distances.append(d)
        node_list.append(label)

    n_distances = np.asarray(distances)
    # log.debug("distances: %s", n_distances)

    # divisor is the maximum possible distance in the region, which is the distance from origin
    # diagonally to the max x and max y coordinate.
    n_total = math.sqrt(max_x_coord**2 + max_y_coord**2)
    frac_distances = n_distances / n_total
    # log.debug("frac distances: %s",frac_distances)

    frac_distances = 1.0 - frac_distances
    # log.debug("inverse frac distances: %s", frac_distances)

    total_frac_distance = np.sum(frac_distances)
    frac_distances = frac_distances / total_frac_distance

    # log.debug("scaled frac distances: %s", frac_distances)

    total_weights = sum(frac_distances)

    return (node_list, frac_distances)


In [44]:
def assign_linear_weighted_edges_to_slice(g, max_x_coord, max_y_coord):
    """
    Takes an empty graph with N nodes, and given parameters for mean/sd node degree, and generates
    random inverse-distance-weighted edges, so that neighbors are preferentially geographically close.

    :param g:
    :return:
    """
    # generate a random number of edges for each vertex, drawn from a lognormal distribution, clipped to 1 on the
    # lower limit (no isolated vertices) and the number of populations minus one on the upper.
    edges_per_vertex = np.clip(np.random.lognormal(1.5,
                                                   0.2, size = 32), 
                               1, 31).astype(np.int64)
    # log.debug("edges per vertex: %s", edges_per_vertex)
    # to allow us to index the number of edges, poor man's generator
    edge_ix = 0
    dist_map = build_distance_map_to_self(g)
    # we need the latter because we usually prefer to deal in node labels, but edge wiring requires IDs.
    label_map = build_map_from_vertex_label_to_id(g)
    dist_stat = calculate_community_distance_statistics(g, ignore_actual=True)

    for v,d in g.nodes_iter(data=True):
        num_neighbors = edges_per_vertex[edge_ix]
        # reduce by the number of existing edges
        num_neighbors -= len(g.neighbors(v))
        if num_neighbors < 1:
            continue

        # log.debug("selecting %s neighbors", num_neighbors)

        n_label = g.node[v]['label']
        other_node_list, weights = build_weighted_node_lists_linear_decay(n_label, dist_map, max_x_coord, max_y_coord)
        # log.debug("other node list: %s", other_node_list)
        # log.debug("weights: %s", weights)

        random_neighbor_list = np.random.choice(other_node_list, size=num_neighbors, p=weights)
        # log.debug("selected neighbors: %s", random_neighbor_list)

        # go over random neighbors and wire them with edges
        for neighbor_label in random_neighbor_list:
            g.add_edge(v, label_map[neighbor_label])

In [43]:

def assign_exponential_weighted_edges_to_slice(g, alpha):
    """
    Takes an empty graph with N nodes, and given parameters for mean/sd node degree, and generates
    random inverse-distance-weighted edges, so that neighbors are preferentially geographically close.

    :param g:
    :return:
    """
    # generate a random number of edges for each vertex, drawn from a lognormal distribution, clipped to 1 on the
    # lower limit (no isolated vertices) and the number of populations minus one on the upper.
    edges_per_vertex = np.clip(np.random.lognormal(1.5,0.2, size = 32), 1, 31).astype(np.int64)
    # log.debug("edges per vertex: %s", edges_per_vertex)
    # to allow us to index the number of edges, poor man's generator
    edge_ix = 0
    dist_map = build_distance_map_to_self(g)
    # we need the latter because we usually prefer to deal in node labels, but edge wiring requires IDs.
    label_map = build_map_from_vertex_label_to_id(g)
    dist_stat = calculate_community_distance_statistics(g, ignore_actual=True)

    for v,d in g.nodes_iter(data=True):
        num_neighbors = edges_per_vertex[edge_ix]
        # reduce by the number of existing edges
        num_neighbors -= len(g.neighbors(v))
        if num_neighbors < 1:
            continue

        # log.debug("selecting %s neighbors", num_neighbors)

        n_label = g.node[v]['label']
        other_node_list, weights = build_weighted_node_lists_exponential_decay(n_label, dist_map, dist_stat['potential_mean'], alpha)
        # log.debug("other node list: %s", other_node_list)
        # log.debug("weights: %s", weights)

        random_neighbor_list = np.random.choice(other_node_list, size=num_neighbors, p=weights)
        # log.debug("selected neighbors: %s", random_neighbor_list)

        # go over random neighbors and wire them with edges
        for neighbor_label in random_neighbor_list:
            g.add_edge(v, label_map[neighbor_label])

In [46]:
def find_maximum_regional_coordinates(aspectratio, numpopulations):
    """
    Given the aspect ratio desired for the regional model (square, long thin, as defined by the ratio of sides),
    and the number of populations, deliver an XY coordinate system (in terms of maxima) that yields a 1% ratio
    of occupied space if each population takes up a 10x10 unit in the coordinate system.

    :param aspectratio:
    :param numpopulations:
    :return: tuple of maximum_x, maximum_y
    """
    occupied_space = 100. * numpopulations
    total_area = occupied_space / 0.01

    side = math.sqrt(total_area / aspectratio)
    x = side
    y = aspectratio * side

    return (x,y)



In [47]:

def build_map_from_vertex_label_to_id(g):
    """
    Keep a map from vertex label to node ID, since we don't want to relabel the nodes in this context

    :param g:
    :return: dict
    """
    label_map = dict()
    for n,d in g.nodes_iter(data=True):
        label_map[g.node[n]['label']] = n

    return label_map

## Trial Analysis ##

In [33]:
(max_x_coord, max_y_coord) = find_maximum_regional_coordinates(8.0, 32)
print max_x_coord
print max_y_coord

200.0
1600.0


In [34]:
g = nx.read_gml("foo-001.gml")

I'm grabbing a regional graph with a 8.0 aspect ratio, meaning that the region is 8x longer than it is wide, to make a long narrow space.  The overall size is governed by ensuring that the 32 populations, if they each take up 10x10 area, make up no more than 1% of the total area of the region.  

First, we make a copy of the graph and remove all the edges, because we're going to rewire it with different distance decay functions.  I do this in a way that leaves node information intact, since that's where the geographic coordinates are.

In [35]:
edge_list = g.edges()
g.remove_edges_from(edge_list)
print calculate_community_distance_statistics(g, ignore_actual=True)

{'potential_mean': 459.66243998708808, 'potential_min': 11.180339887498949, 'potential_max': 1523.4992615685771}


These are the reference values for how far away graph nodes are:  at a minimum, about 11 units, max about 1523, with an average of about 460.  Let's see how we can get **wired** edges that yield an actual network with smaller distances between the wired nodes.  

### Linear Distance Decay ###

In [41]:
linear_g = g.copy()

In [48]:
assign_linear_weighted_edges_to_slice(linear_g, max_x_coord, max_y_coord)

In [51]:
print calculate_community_distance_statistics(linear_g, ignore_actual=False)

{'edge_density': 0.11693548387096774, 'potential_max': 1523.4992615685771, 'actual_mean': 452.63333549773301, 'actual_max': 1315.8632147757608, 'potential_mean': 459.66243998708808, 'actual_min': 30.083217912982647, 'potential_min': 11.180339887498949}


As we can see, the mean distance isn't much smaller than the potential mean distance, so we haven't really shrunk the locality of the graph yet.  

### Exponential Decay with Scale Factor ###

In [62]:
exponential_1 = g.copy()
assign_exponential_weighted_edges_to_slice(exponential_1, 1.0)
print calculate_community_distance_statistics(exponential_1, ignore_actual=False)

{'edge_density': 0.18548387096774194, 'potential_max': 1523.4992615685771, 'actual_mean': 321.16227979969773, 'actual_max': 1184.7193760549374, 'potential_mean': 459.66243998708808, 'actual_min': 11.180339887498949, 'potential_min': 11.180339887498949}


In [58]:
exponential_2 = g.copy()
assign_exponential_weighted_edges_to_slice(exponential_2, 2.0)
print calculate_community_distance_statistics(exponential_2, ignore_actual=False)

{'edge_density': 0.1592741935483871, 'potential_max': 1523.4992615685771, 'actual_mean': 254.53850706974896, 'actual_max': 1140.8492450801728, 'potential_mean': 459.66243998708808, 'actual_min': 22.0, 'potential_min': 11.180339887498949}


In [60]:
exponential_3 = g.copy()
assign_exponential_weighted_edges_to_slice(exponential_3, 3.0)
print calculate_community_distance_statistics(exponential_3, ignore_actual=False)

{'edge_density': 0.16532258064516128, 'potential_max': 1523.4992615685771, 'actual_mean': 183.80658176230142, 'actual_max': 636.94426757762722, 'potential_mean': 459.66243998708808, 'actual_min': 11.180339887498949, 'potential_min': 11.180339887498949}


In [61]:
exponential_4 = g.copy()
assign_exponential_weighted_edges_to_slice(exponential_4, 4.0)
print calculate_community_distance_statistics(exponential_4, ignore_actual=False)

{'edge_density': 0.1975806451612903, 'potential_max': 1523.4992615685771, 'actual_mean': 168.56577527337745, 'actual_max': 697.20728624993581, 'potential_mean': 459.66243998708808, 'actual_min': 11.180339887498949, 'potential_min': 11.180339887498949}
